In [207]:
import pandas as pd
pd.set_option('display.max_columns', None)
import censusdata as cd
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd
import os
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import contextily as cx
from sridentify import Sridentify
import statistics
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)
plt.style.use('seaborn-dark')
from matplotlib.backends.backend_pdf import PdfPages

us_state_abbrev = {'the United States':'US','Alabama': 'AL','Alaska': 'AK','American Samoa': 'AS','Arizona': 'AZ',
'Arkansas': 'AR','California': 'CA','Colorado': 'CO', 'Connecticut': 'CT','Delaware': 'DE','District of Columbia': 'DC',
 'Florida': 'FL','Georgia': 'GA','Guam': 'GU','Hawaii': 'HI', 'Idaho': 'ID','Illinois': 'IL','Indiana': 'IN',
 'Iowa': 'IA','Kansas': 'KS','Kentucky': 'KY','Louisiana': 'LA', 'Maine': 'ME','Maryland': 'MD','Massachusetts': 'MA',
 'Michigan': 'MI','Minnesota': 'MN','Mississippi': 'MS','Missouri': 'MO', 'Montana': 'MT','Nebraska': 'NE','Nevada': 'NV',
 'New Hampshire': 'NH','New Jersey': 'NJ','New Mexico': 'NM','New York': 'NY','North Carolina': 'NC', 
 'North Dakota': 'ND','Northern Mariana Islands':'MP','Ohio': 'OH','Oklahoma': 'OK','Oregon': 'OR',
 'Pennsylvania': 'PA','Puerto Rico': 'PR', 'Rhode Island': 'RI','South Carolina': 'SC','South Dakota': 'SD',
 'Tennessee': 'TN','Texas': 'TX','Utah': 'UT', 'Vermont': 'VT','Virgin Islands': 'VI','Virginia': 'VA',
 'Washington': 'WA','West Virginia': 'WV','Wisconsin': 'WI','Wyoming': 'WY'}
abbrev_us_state = dict(map(reversed, us_state_abbrev.items()))

state_codes = {'WA': '53', 'DE': '10', 'DC': '11', 'WI': '55', 'WV': '54', 'HI': '15','FL': '12', 'WY': '56', 'PR': '72', 'NJ': '34', 'NM': '35', 'TX': '48',
    'LA': '22', 'NC': '37', 'ND': '38', 'NE': '31', 'TN': '47', 'NY': '36','PA': '42', 'AK': '02', 'NV': '32', 'NH': '33', 'VA': '51', 'CO': '08',
    'CA': '06', 'AL': '01', 'AR': '05', 'VT': '50', 'IL': '17', 'GA': '13','IN': '18', 'IA': '19', 'MA': '25', 'AZ': '04', 'ID': '16', 'CT': '09',
    'ME': '23', 'MD': '24', 'OK': '40', 'OH': '39', 'UT': '49', 'MO': '29','MN': '27', 'MI': '26', 'RI': '44', 'KS': '20', 'MT': '30', 'MS': '28',
    'SC': '45', 'KY': '21', 'OR': '41', 'SD': '46'}

state_to_crs = {'WA': 'EPSG:2285', 'DE': 'EPSG:2235', 'DC': 'EPSG:2283', 'WI': 'EPSG:2288', 'WV': 'EPSG:2857', 'HI': 'EPSG:2786','FL': 'EPSG:2778', 'WY': 'EPSG:2863', 
               'PR': 'EPSG:2866', 'NJ': 'EPSG:2824', 'NM': 'EPSG:2258', 'TX': 'EPSG:2277','LA': 'EPSG:2801', 'NC': 'EPSG:2264', 
               'ND': 'EPSG:2266', 'NE': 'EPSG:2819','TN': 'EPSG:2204', 'NY': 'EPSG:2261','PA': 'EPSG:2272', 'AK': 'EPSG:3467', 
                'NV': 'EPSG:2821', 'NH': 'EPSG:2823', 'VA': 'EPSG:2284', 'CO': 'EPSG:2232','CA': 'EPSG:2769', 'AL': 'EPSG:2759',  'AR': 'EPSG:2765', 'VT': 'EPSG:2852', 'IL': 'EPSG:2790', 
               'GA': 'EPSG:2239','IN': 'EPSG:2244','IA': 'EPSG:2795', 'MA': 'EPSG:2249','AZ': 'EPSG:2223', 'ID': 'EPSG:2242', 'CT': 'EPSG:2775','ME': 'EPSG:2802', 'MD': 'EPSG:2804', 'OK': 'EPSG:2268', 
               'OH': 'EPSG:2835', 'UT': 'EPSG:2280','MO': 'EPSG:2816','MN': 'EPSG:2811', 'MI': 'EPSG:2252', 'RI': 'EPSG:2840', 'KS': 'EPSG:2796', 'MT': 'EPSG:2256', 
               'MS': 'EPSG:2254','SC': 'EPSG:2273', 'KY': 'EPSG:2205', 'OR': 'EPSG:2991', 'SD': 'EPSG:2266'}

states = ["AL", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", "MA", "MI", "MN", "MS", 
          "MO", "MT", "NE", "NV", "NH", "NJ", "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", 
          "RI", "SC", "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

In [3]:
ct = gpd.GeoDataFrame(gpd.read_file('../Static/GIS_Data/cb_2019_us_tract_500k/cb_2019_us_tract_500k.shp'))
ct.crs = 'EPSG:4269'

In [70]:
def merge_df_ct(sa):
    ct2 = ct[ct.STATEFP==state_codes[sa]]
    df = pd.read_csv('../Static/NREL LEAD/Tracts/Energy Expenditures/'+sa+'_NREL_LEAD_Energy_Expenditures.csv',index_col=0)
    fips = list(ct2.GEOID.unique())
    geod = dict()
    for fip in fips:
        try:
            df2 = df[df.FIP==int(fip)].reset_index(drop=True)
            geod[str(fip)] = list(df2.iloc[0,1:])
        except:
            geod[str(fip)] = [-1,0,0,0,0,0,0]
    ct2.loc[:,'UNITS'] = [geod[ct2.loc[i,'GEOID']][0] for i in ct2.index]
    ct2.loc[:,'HINCP'] = [geod[ct2.loc[i,'GEOID']][1] for i in ct2.index]
    ct2.loc[:,'NRG_EXP'] = [geod[ct2.loc[i,'GEOID']][2] for i in ct2.index]
    ct2.loc[:,'NRG_BURD'] = [geod[ct2.loc[i,'GEOID']][3] for i in ct2.index]
    ct2.loc[:,'ELEP'] = [geod[ct2.loc[i,'GEOID']][4] for i in ct2.index]
    ct2.loc[:,'GASP'] = [geod[ct2.loc[i,'GEOID']][5] for i in ct2.index]
    ct2.loc[:,'FULP'] = [geod[ct2.loc[i,'GEOID']][6] for i in ct2.index]
    ct2 = ct2[ct2.UNITS>=0].reset_index(drop=True)
    try:
        ct2.to_crs(state_to_crs[sa], inplace=True)
    except:
        None
    return ct2

In [228]:
def save_expenditure_figures(df, sa):
    x = 10
    plot_num = 4
    fig, ax = plt.subplots(plot_num, figsize=(x,x*plot_num))
    df[df.NRG_BURD<0.15].plot(ax=ax[0], cmap='Spectral_r', column='NRG_BURD',
           legend=True, 
           legend_kwds={'orientation': "horizontal", "label":"\nEnergy Expenditure as % of Income"})
    df[df.NRG_BURD>=0.2].plot(ax=ax[0], color='#c90606')
    ax[0].axis('off')
    ax[0].set_title('Energy Burden in '+ abbrev_us_state[sa] + ', 2016', fontsize=20)

    #fig2, ax2 = plt.subplots(1, figsize=(10,10))
    df.plot(ax=ax[1], cmap='Spectral_r', column='ELEP',
           legend=True, 
           legend_kwds={'orientation': "horizontal", "label":"\nAverage Household Electricity Expenditures ($)"})
    ax[1].axis('off')
    ax[1].set_title('\n\n\n Electricity Expenditures in '+ abbrev_us_state[sa] + ', 2016', fontsize=20)
    fig.tight_layout()

    df.plot(ax=ax[2], cmap='Spectral_r', column='GASP',
           legend=True, 
           legend_kwds={'orientation': "horizontal", "label":"\nAverage Household Natural Gas Expenditures ($)"})
    ax[2].axis('off')
    ax[2].set_title('\n\n\n Natural Gas Expenditures in '+ abbrev_us_state[sa] + ', 2016', fontsize=20)
    fig.tight_layout()
    
    df.plot(ax=ax[3], cmap='Spectral_r', column='FULP',
           legend=True, 
           legend_kwds={'orientation': "horizontal", "label":"\nAverage Household Fuel Oil Expenditures ($)"})
    ax[3].axis('off')
    ax[3].set_title('\n\n\n Fuel Oil Expenditures in '+ abbrev_us_state[sa] + ', 2016', fontsize=20)
    fig.tight_layout()
    
    plt.savefig('../Visualizations/States/'+abbrev_us_state[sa]+'/' + abbrev_us_state[sa] + ' Energy Expenditure Maps.pdf')
    plt.close()

In [227]:
for sa in states:
    df = merge_df_ct(sa)
    save_expenditure_figures(df, sa)

/Users/ryaniyer/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/ryaniyer/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/Users/ryaniyer/opt/anaconda3/lib/python3.7/site-packages/geopandas/plotting.py:532: UserWarning: The GeoDataFrame you are attempting to plot is empty. Nothing has been displayed.
  User

/Users/ryaniyer/opt/anaconda3/lib/python3.7/site-packages/geopandas/plotting.py:532: UserWarning: The GeoDataFrame you are attempting to plot is empty. Nothing has been displayed.
  UserWarning,
/Users/ryaniyer/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/ryaniyer/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[i

/Users/ryaniyer/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/ryaniyer/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/Users/ryaniyer/opt/anaconda3/lib/python3.7/site-packages/geopandas/plotting.py:532: UserWarning: The GeoDataFrame you are attempting to plot is empty. Nothing has been displayed.
  User

/Users/ryaniyer/opt/anaconda3/lib/python3.7/site-packages/geopandas/plotting.py:532: UserWarning: The GeoDataFrame you are attempting to plot is empty. Nothing has been displayed.
  UserWarning,
/Users/ryaniyer/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/ryaniyer/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[i

/Users/ryaniyer/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/ryaniyer/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/Users/ryaniyer/opt/anaconda3/lib/python3.7/site-packages/geopandas/plotting.py:532: UserWarning: The GeoDataFrame you are attempting to plot is empty. Nothing has been displayed.
  User

/Users/ryaniyer/opt/anaconda3/lib/python3.7/site-packages/geopandas/plotting.py:532: UserWarning: The GeoDataFrame you are attempting to plot is empty. Nothing has been displayed.
  UserWarning,
/Users/ryaniyer/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/ryaniyer/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[i